# **AI-Powered Activity Suggestion System**
This notebook suggests real-world activities based on user interests and meeting location using the **Google Places API** and **Sentence Transformers** for AI-based matching.

In [2]:
# Install required packages (uncomment if not installed)
#!pip install googlemaps sentence-transformers numpy scikit-learn

In [3]:
import googlemaps
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# Load an embedding model for matching interests
model = SentenceTransformer('all-MiniLM-L6-v2')

# Google Places API Key (Replace with your key)
GOOGLE_API_KEY = "Replace with your API key"  # 
gmaps = googlemaps.Client(key=GOOGLE_API_KEY)

/opt/anaconda3/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
def get_real_world_activities(location, radius=5000, activity_type="tourist_attraction"):
    """
    Fetches nearby activities using Google Places API based on location and activity type.
    """
    places = gmaps.places_nearby(location=location, radius=radius, type=activity_type)
    
    if "results" in places:
        return {place["name"]: place["types"] for place in places["results"]}
    
    return {}

In [5]:
def suggest_activity(user_interests, location):
    """
    Matches user interests with real-world activities and suggests the best match.
    """
    activities = get_real_world_activities(location)
    if not activities:
        return "No activities found nearby. Try a different location or expand the radius."

    activity_names = list(activities.keys())
    activity_descriptions = [" ".join(activities[name]) for name in activity_names]

    # Convert descriptions to vector embeddings
    activity_vectors = model.encode(activity_descriptions)
    user_vector = model.encode([user_interests])

    # Compute similarity scores
    scores = cosine_similarity(user_vector, activity_vectors)[0]
    best_match = np.argmax(scores)

    return activity_names[best_match]  # Return best-matching real-world activity

## **Test the Activity Suggestion System**
Provide user interests and a meeting location (latitude, longitude) to get an activity recommendation.

In [7]:
# Example usage
user_interests = "I love outdoor adventures and nature walks"
location = "40.748817,-73.985428"  # Example: Times Square, NYC (lat,long)
suggestion = suggest_activity(user_interests, location)

print(f"Suggested Activity: {suggestion}")

Suggested Activity: Washington Square Park
